### Grouper
pd.Grouper()是对DataFrame.groupby()的一个增强操作，格式大致为DataFrame.groupby([pd.Grouper()])，其地位等价于指定一个groupby的名字，如下面写法也是可以的DataFrame.groupby([df['var1'], pd.Grouper()])。另外，当指定freq参数时，相当于联合使用了groupby和resample，这种用法可以解决groupby和resample联合后使用.agg([])会出错的bug。

In [1]:
# 这里用和《pandas_agg》一样的原始数据
import pandas as pd

df = pd.read_excel("https://github.com/chris1610/pbpython/blob/master/data/sample-salesv3.xlsx?raw=True")  # 从网络read_excel
df["date"] = pd.to_datetime(df['date'])  # 将"date"转为pandas里的datetime类型(为了能够resample)
print(df.head())

   account number                         name       sku  quantity  \
0          740150                   Barton LLC  B1-20000        39   
1          714466              Trantow-Barrows  S2-77896        -1   
2          218895                    Kulas Inc  B1-69924        23   
3          307599  Kassulke, Ondricka and Metz  S1-65481        41   
4          412290                Jerde-Hilpert  S2-34077         6   

   unit price  ext price                date  
0       86.69    3380.91 2014-01-01 07:21:51  
1       63.16     -63.16 2014-01-01 10:00:47  
2       90.70    2086.10 2014-01-01 13:24:58  
3       21.05     863.05 2014-01-01 15:05:22  
4       83.21     499.26 2014-01-01 23:26:55  


In [2]:
# 在groupby里面用Grouper
df11 = df.groupby(pd.Grouper(key='name'))  # 两种groupby方法等价
df12 = df.groupby('name')
print('两种groupby方法是等价的')
print(f'df11: {df11}')
print(f'df12: {df12}')

# 用pd.Grouper来解决groupby/resample联合使用时的bug（具体可见《pandas_agg》的第2点）
def get_max(x):
    return x.value_counts(dropna=False).index[0]

print('注意：用Grouper可以解决groupby\\resample联合时，agg([])会出错的bug——但是groupby\\rolling暂时无法解决')
df2 = df.groupby(['name', pd.Grouper(key='date', freq='M')])[['unit price', 'ext price', 'sku']].\
      agg({'ext price': ['sum', 'mean'], 'unit price': ['mean'], 'sku': [get_max]})
print(df2)

两种groupby方法是等价的
df11: <pandas.core.groupby.groupby.DataFrameGroupBy object at 0x02C50CD0>
df12: <pandas.core.groupby.groupby.DataFrameGroupBy object at 0x02C50B10>
注意：用Grouper可以解决groupby\resample联合时，agg([])会出错的bug——但是groupby\rolling暂时无法解决
                                           ext price              unit price  \
                                                 sum         mean       mean   
name                            date                                           
Barton LLC                      2014-01-31   6177.57   772.196250  52.787500   
                                2014-02-28  12218.03  1745.432857  65.202857   
                                2014-03-31   3513.53  1171.176667  52.453333   
                                2014-04-30  11474.20  1147.420000  49.400000   
                                2014-05-31  10220.17  1277.521250  51.661250   
                                2014-06-30  10463.73  1307.966250  51.691250   
                                2014-07-3